In [ ]:
!pip3 install -U opencv-python

In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

In [ ]:
cap = cv2.VideoCapture(0)
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
fgbg = cv2.createBackgroundSubtractorKNN()
#createBackgroundSubtractorGMG()
while(1):
    ret, frame1 = cap.read()
    ret, frame2 = cap.read()
    if not ret:
        print("problem")
        continue
    gray = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    fgmask = fgbg.apply(frame1)
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)
    # Вместо маски будем показывать исходное видео в оттенках серого.
    gray2 = np.where(fgmask==255, gray, np.zeros(gray.shape, np.uint8))
    # Поиск движущихся объектов
    diff = cv2.absdiff(frame1, frame2) #абсолютная разница между элементами матриц
    gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY) #конвертация из RGB в оттенки серого
    blur = cv2.GaussianBlur(gray, (5, 5), 0) #размытие картинки в оттенках серого

    dilated = cv2.dilate(blur, None, iterations=3)
    _ , thresh = cv2.threshold(dilated, 20, 255, cv2.THRESH_BINARY)

    cntrs, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    for cntr in cntrs:
        if cv2.contourArea(cntr) < 500:
            continue
        (x, y, w, h) = cv2.boundingRect(cntr)
        cv2.rectangle(frame1, (x, y), (x + w, y + h), (0, 255, 0), 2) #frame1
    cv2.imshow('frame', gray2)
    cv2.imshow('original', frame1)
    if cv2.waitKey(1) & 0xff == 27:
        break
cap.release()
cv2.destroyAllWindows()

In [ ]:
cap = cv2.VideoCapture(0)
# Ядро для свертки изображения.
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
fgbg = cv2.createBackgroundSubtractorKNN()
#fgbg = cv2.createBackgroundSubtractorGMG()
cntr = 0

while(1):
    cntr += 1
    if cntr == 100:
        fgbg = cv2.createBackgroundSubtractorKNN()
        cntr = 0

    ret, frame = cap.read()
    # Получаем маску, ...
    fgmask = fgbg.apply(frame)
    # ... сворачиваем ее и отображаем.
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)
    cv2.imshow('frame', fgmask)
    if cv2.waitKey(3) & 0xff == 27:
        break
cap.release()
cv2.destroyAllWindows()